In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

#openai.api_key = os.environ["OPENAI_API_KEY"]


Created company www.amazon.com : 1707858349473x785784134804136600
Retrieved project Amazon_trustpilot : 1707841778067x515781546632932540


In [2]:
prompts_path = "Prompts/en/"

In [3]:
with open(prompts_path+'prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Business context.")

    def __str__(self):
        return self.context

res = get_analysis(prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Amazon.com is an American multinational technology company focusing on e-commerce, cloud computing, digital streaming, and artificial intelligence. It is one of the world's largest online marketplaces, AI assistants providers, and cloud computing platforms. Recently, Amazon has been expanding its presence in healthcare, grocery, and advertising sectors. Some of its competitors include Walmart, Microsoft, Google, and Netflix.


In [ ]:
# SEND IT TO BUBBLE

In [4]:
with open(prompts_path+'prompt_categories_creation.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)

class InsightSubCategory(BaseModel):
    name: str = Field(description="Subcategory name.")
    description: str = Field(description="Description sentence for the subcategory.")
    id: str
    def __str__(self):
        return '\n- ' + self.name + ": " + self.description

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    #description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[InsightSubCategory] = Field(description="Subcategory.")
    id: str

    def __str__(self):
        return self.name + " : " + ''.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Categories into which the insights can be classified.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(prompt_categories_creation, InsightCategories)
print(res)

Overall Satisfaction : 
- Overall Rating: Indicates the customer's overall satisfaction level with Amazon.
- General Comment: Allows the customer to express themselves freely about their experience with Amazon.

Customer Service Experience : 
- Customer Service Effectiveness: Evaluates the effectiveness of Amazon's customer service in answering questions and resolving customer problems.
- Customer Service Friendliness: Rates the friendliness and helpfulness of Amazon's customer service.

Website and Mobile Application : 
- Ease of Use: Indicates whether Amazon's website and mobile app are easy to use.
- Features: Evaluates customer satisfaction with website and mobile app features.
- Speed of Delivery: Evaluates the speed of delivery of orders placed on Amazon's website or mobile app.

Product Quality : 
- Product Freshness: Evaluates the freshness of fresh and perishable products sold on Amazon.
- Product Variety: Indicates whether the choice of products on Amazon is sufficient and me

In [6]:
for categorie in tqdm(res.categories):
    categorie.id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
        })
    
    for sub_categorie in categorie.sub_categories:
        sub_categorie.id = bubble_client.create(
            "SubCategory",
               { 
                "Company": COMPANY_ID,
                "Parent_category": categorie.id,
                "Name": sub_categorie.name,
                "Project": PROJECT_ID,
                "Description": sub_categorie.description,}
            )

  0%|          | 0/7 [00:00<?, ?it/s]